In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
import nltk

In [3]:
data = pd.read_csv('movie_dataset/train.csv')

data.head()

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0


In [4]:
data.isnull().sum() #결측치 없음

id          0
document    0
label       0
dtype: int64

In [5]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(data)
train.reset_index(inplace=True)
val.reset_index(inplace=True)

In [6]:
train

,index,id,document,label
0,3991,3992,너무유치하다진짜 여기저기서짜집기해오고,0
1,1467,1468,스마트폰 뒀다뭐할래? 초행길이라 잘 주웠다고? 흔한 어플하나 안깔고 뭐하냐,0
2,4666,4667,괜히 봤다...정말....진짜 이게 영화? 뭥미?,0
3,3264,3265,마지막 장면이 왜 이렇게 가슴이 아플까요...,1
4,2927,2928,신인시절이라 그런가 정우성의 연기가 ....,0
...,...,...,...,...
3745,2304,2305,이도저도 아닌 어설픈 영화.. 장미인애는 기본적인 대사전달도 안되고..,0
3746,2157,2158,아주 가끔 웃기는 것 빼고는 그냥 평범한 킬링타임용,0
3747,1537,1538,점수 평균 만들려고 10점 주고 감,1
3748,2934,2935,별로.. 1시간 보다가 지루함,0


In [7]:
val

,index,id,document,label
0,3633,3634,정말 최악의 영화ㅎ아ㅜㅜ착하게 말못하겠다. 걍 최악의 쓰레기삼류OO영화임.,0
1,2624,2625,감독님 대체 뭘전달해주시려는거죠?,0
2,325,326,"처음본순간 충격, 평생 잊혀지지 않는 영화.",1
3,28,29,"카모메,안경은 재밌었는데..이건 지루하다..",0
4,1339,1340,리차드 기어... 정말 멋있다.. 주제곡도 최고 ㅎㅎ,1
...,...,...,...,...
1245,2569,2570,진짜 잘 만든 영화. 제한된 장소인데도 엄청난 긴장감이 있다.,1
1246,675,676,명작. 이게 바로 프랑스 영화인가 싶다 여운이 길게 남는다,1
1247,4683,4684,골룸에겐 아쉬운데로 리벨룽겐의 반지를 ㅋ ㅋ ㅋ ㅋ ㅋ ㅋ ㅋ ㅋ ㅋ ㅋ ㅋ,1
1248,1115,1116,무협지에서 자주 등장하던 복수의 악순환. 분위기부터 구성까지 아주 좋았음.,1


In [8]:
#품사태깅 후 임의로 품사를 가져와 긍/부정을 판단할 수 있도록 데이터 전처리
from konlpy.tag import Okt

okt = Okt()
print(okt.pos('오늘도 열심히 재미있는')) #test

[('오늘', 'Noun'), ('도', 'Josa'), ('열심히', 'Adverb'), ('재미있는', 'Adjective')]


In [9]:
main_pos = []
for sentence in train['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
train['main_pos'] = pd.DataFrame(main_pos)
train.head()
    

,index,id,document,label,main_pos
0,3991,3992,너무유치하다진짜 여기저기서짜집기해오고,0,너무 유치하다 진짜 여기저기 짜집기 해오고
1,1467,1468,스마트폰 뒀다뭐할래? 초행길이라 잘 주웠다고? 흔한 어플하나 안깔고 뭐하냐,0,스마트폰 뒀다 뭐 할래 초행길 잘 주웠다고 흔한 어플 하나 깔고 뭐 하냐
2,4666,4667,괜히 봤다...정말....진짜 이게 영화? 뭥미?,0,괜히 봤다 정말 진짜 이 영화 뭥미
3,3264,3265,마지막 장면이 왜 이렇게 가슴이 아플까요...,1,마지막 장면 왜 이렇게 가슴 아플까 요
4,2927,2928,신인시절이라 그런가 정우성의 연기가 ....,0,신인시절 런가 정우성 연기


In [10]:
#CountVectorzier 사용
X_train = train.main_pos
y_train = train.label

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(X_train)
X_train_vec = vectorizer.transform(X_train)


In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier()
model.fit(X_train_vec, y_train)

RandomForestClassifier()

In [13]:
main_pos = []
for sentence in val['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
val["main_pos"] = pd.DataFrame(main_pos)

val 

,index,id,document,label,main_pos
0,3633,3634,정말 최악의 영화ㅎ아ㅜㅜ착하게 말못하겠다. 걍 최악의 쓰레기삼류OO영화임.,0,정말 최악 영화 착하게 말못하겠다 걍 최악 쓰레기 삼류 영화 임
1,2624,2625,감독님 대체 뭘전달해주시려는거죠?,0,감독 대체 뭘 전달 해주시려는 거
2,325,326,"처음본순간 충격, 평생 잊혀지지 않는 영화.",1,처음 순간 충격 평생 잊혀지지 않는 영화
3,28,29,"카모메,안경은 재밌었는데..이건 지루하다..",0,카 메 안경 재밌었는데 이건 지루하다
4,1339,1340,리차드 기어... 정말 멋있다.. 주제곡도 최고 ㅎㅎ,1,리차드 기어 정말 멋있다 주제곡 최고
...,...,...,...,...,...
1245,2569,2570,진짜 잘 만든 영화. 제한된 장소인데도 엄청난 긴장감이 있다.,1,진짜 잘 만든 영화 제한 된 장소 도 엄청난 긴장감 있다
1246,675,676,명작. 이게 바로 프랑스 영화인가 싶다 여운이 길게 남는다,1,명작 이 바로 프랑스 영화 싶다 여운 길게 남는다
1247,4683,4684,골룸에겐 아쉬운데로 리벨룽겐의 반지를 ㅋ ㅋ ㅋ ㅋ ㅋ ㅋ ㅋ ㅋ ㅋ ㅋ ㅋ,1,골룸 아쉬운데로 리벨룽겐 반지
1248,1115,1116,무협지에서 자주 등장하던 복수의 악순환. 분위기부터 구성까지 아주 좋았음.,1,무협지 자주 등장 하던 복수 악순환 분위기 구성 아주 좋았음


In [16]:
X_val = val.main_pos
y_val = val.label

X_val_vec = vectorizer.transform(X_val)

In [17]:
y_pred = model.predict(X_val_vec)
print(y_pred)

[0 0 1 ... 1 1 1]


In [18]:
from sklearn import metrics

print('accuracy=', metrics.accuracy_score(y_val, y_pred))

accuracy= 0.7888


In [19]:
test = pd.read_csv('movie_dataset/test.csv')
test

,id,document
0,1,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!
2,3,Childhood fantasy
3,4,멋있는 영화입니다. 잊을 수 없는!
4,5,너무 감동적이네요 펑펑 울었습니다
...,...,...
4995,4996,연계도 제대로 안되고 뭔가 부족한느낌이들고 찝찝한영화
4996,4997,내생에 최악의 영화중 하나 졸려서 미칠뻔....
4997,4998,지금까지 나온 드라마 중 최고였습니다. 100점 만점!
4998,4999,영화값으로 엄청난 공연을 보았다 - 이게 웬 횡재 ..


In [20]:
test.isnull().sum()

id          0
document    0
dtype: int64

In [21]:
main_pos = []
for sentence in test['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun","Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
test["main_pos"] = pd.DataFrame(main_pos)

test.head()

,id,document,main_pos
0,1,시간 때우기 좋은 영화 지루함,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!,훈훈한 정이 느껴지는 영화 가족 끼리 드라마 보듯이 보면 딱
2,3,Childhood fantasy,
3,4,멋있는 영화입니다. 잊을 수 없는!,멋있는 영화 입니다 잊을 수 없는
4,5,너무 감동적이네요 펑펑 울었습니다,너무 감동 적이네요 펑펑 울었습니다


In [22]:
X_test = test.main_pos
X_test_vec = vectorizer.transform(X_test)
pred_test = model.predict(X_test_vec)
print(pred_test)

[0 0 1 ... 1 0 1]


In [23]:
submission = pd.read_csv('movie_dataset/sample_submission.csv')
submission.head()

,id,label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [24]:
submission['label'] = pred_test

submission

,id,label
0,1,0
1,2,0
2,3,1
3,4,1
4,5,1
...,...,...
4995,4996,0
4996,4997,0
4997,4998,1
4998,4999,0


In [25]:
submission.to_csv("submission_mycode_rfc.csv", index=False)